In [248]:
import numpy as np
import math


# List of basic ship information

loa = 158 # ship overall length
lwl = 144.8 # ship waterline length, m
lpp = 0.97*lwl # ship perpendicular length , m, according to information
B = 24.5 # Ship B, m
depth = 13.8 # Ship depth. m
T_n = 5.85 # Nominal max draught , m
dwt = 5110 # ship dead weight , t
V_n = 17.7 # ship design speed, knots

#list of constants

g = 9.805 # gravity, kg/ms^2 
rho_sea = 1025 # kg/m3
nu_sea = 0.00000118 # Dynamic viscosity of sea m^2/s

In [249]:
# Calculation for C_b according to Schneekluth and Bertram 1998
# first calculate froude number Fn
V_n = 17.7/1.94384
Fr_n = V_n / math.sqrt(g*lpp)
print(f"Froude Number {Fr_n:0.4f}")
C_b = -4.22 + 27.8*math.sqrt(Fr_n) - 39.1*Fr_n + 46.6*(Fr_n)**3
print(f"C_b {C_b:0.4f}")


Froude Number 0.2454
C_b 0.6451


In [250]:
# calculation for C_m according to charchalis 2017

C_m = 0.977 + 0.085*(C_b-0.6)
print(f"C_m {C_m:0.4f}")

# prismatic coefficient C_p can be calculated

C_p = C_b/C_m 
print(f"C_p {Fr_n:0.4f}")


C_m 0.9808
C_p 0.2454


In [251]:
# Displacement calculation according to barras
# Use approximate value cd=0.35 according to Barras

dsp = dwt/0.35 # m^3
print(f"dsp {dsp:0.4f} m^3")


dsp 14600.0000 m^3


In [252]:
# coefficient c14 to account for stern shape according to holtrop mennen

C_stern = 10 # assume u shaped stern
c14 = 1 + 0.011*C_stern 
print(f"c14 {c14:0.4f}")


c14 1.1100


In [253]:
# Calculate length of run according to holtrop mennen

lcb = -2/100 # ratio lcb wrt to amidship ford, value from barras
# L in holtrop mennen is lwl
lr = lwl*(1-C_p+(0.06*C_p*lcb/(4*C_p-1)))
print(f"lr {lr:0.4f} m")


lr 49.4890 m


In [254]:
# now the (1+k1) can be calculated

k1a = 0.487118*c14*(B/lwl)**1.06806
k1b = (T_n/lwl)**0.46106
k1c = (lwl/lr)**0.121563
k1d = (lwl**3/dsp)**0.36486
k1e = (1-C_p)**-0.604247

k1_const = 0.93 + k1a*k1b*k1c*k1d*k1e

print(f"k1_const {k1_const:0.4f}")


k1_const 1.2119


In [255]:
# we can then calculate R_f
import pandas as pd
import numpy as np
# use v mean = 10 for Reynold

v_dummy = [17,18,19,20,21]
v_dummy_df = pd.DataFrame(v_dummy,columns=['dummy_vel'])
v_dummy_df['dummy_vel_ms'] = v_dummy_df['dummy_vel'] / 1.94384
v_dummy_df['Re'] =( v_dummy_df['dummy_vel_ms'] * lwl ) / nu_sea
v_dummy_df['cf'] = 0.075 / (np.log10(v_dummy_df['Re']-2)**2)
print(v_dummy_df['Re'])
print(v_dummy_df['cf'])

# calculate appendage area of bare hull S_bh

# first calculate C_wp acc. to schneekluth and bertram
C_wp = (1+2*C_b)/3
print(f"C_wp {C_wp:0.4f}")

# A_bt according to kim

A_m = B*T_n*C_m
A_t = 0.051 * A_m
A_bt = 0.08*A_m
print(A_bt)

print(f"A_bt {A_bt:0.4f} m^2")

sbh_a = lwl*(2*T_n+B)*math.sqrt(C_m)
sbh_b = 0.453
sbh_c = 0.4425*C_b
sbh_d = 0.2862*C_m
sbh_e = 0.003467*(B/T_n)
sbh_f = 0.3696*C_wp
sbh_g = 2.38*A_bt/C_b

S_bh = sbh_a*(sbh_b+sbh_c-sbh_d+sbh_e+sbh_f)+sbh_g
print(f"S_bh {S_bh:0.4f} m^2")

v_dummy_df['R_f'] = 0.5 * rho_sea * (v_dummy_df['dummy_vel_ms'])**2 * v_dummy_df['cf'] * S_bh * k1_const
print(v_dummy_df['R_f'])



0    1.073186e+09
1    1.136314e+09
2    1.199443e+09
3    1.262572e+09
4    1.325700e+09
Name: Re, dtype: float64
0    0.000920
1    0.000915
2    0.000910
3    0.000905
4    0.000901
Name: cf, dtype: float64
C_wp 0.7634
11.24627236597615
A_bt 11.2463 m^2
S_bh 3958.0019 m^2
0    172913.623194
1    192793.359170
2    213700.189821
3    235629.204829
4    258575.778420
Name: R_f, dtype: float64


In [256]:
# Now resistance due to appendage
# Assume Sapp

S_app = 50 # m^2 
# from holtrop mennen
k2_const = 2.8

v_dummy_df['Rapp'] = 0.5 * rho_sea * (v_dummy_df['dummy_vel_ms'])**2 * (v_dummy_df['cf']) * S_app *k2_const
print(v_dummy_df['Rapp'])


0    5046.840273
1    5627.071317
2    6237.280235
3    6877.323709
4    7547.066726
Name: Rapp, dtype: float64


In [257]:
print(B/lwl)
print(lwl/B)
print(lwl**3/dsp)

0.1691988950276243
5.910204081632654
207.94708164383565


In [258]:
# calculate wave resistance
c7 = B/lwl
T_fwd = T_n # assume !
h_b = 0.6*T_fwd
c3 = 0.56 * A_bt**1.5 / (B*T_n*(0.31*math.sqrt(A_bt)+T_fwd-h_b))
print(c3)
c2 = math.exp(-1.89*math.sqrt(c3))
print(c2)
c5 = 1 - 0.8*(A_t/(B*T_n*C_m))
print(c5)
lambda_const = (1.446 * C_p) - 0.03*(lwl/B)
print(lambda_const)
c16 = 8.07981*C_p - 13.8673*C_p**2 + 6.984388*C_p**3
print(c16)
m_1 = 0.0140407 * (lwl/T_n) - 1.75254*(dsp**(1/3)/lwl) -  4.79323*(B/lwl) - c16
print(m_1)
c15 = -1.69385
m_2 = c15 * C_p**2 * math.exp(-0.1*Fr_n**-2)
print(m_2)
i_e = 1 + 89*math.exp(-(lwl/B)**0.80856*(1-C_wp)**0.30484*(1-C_p-0.0225*lcb)**0.6367*(lr/B)**0.34574*((100*dsp)/lwl**3)**0.16302)
print(i_e)
c1 = 2223105 * c7**3.78613 * (T_n/B)**1.07961*(90-i_e)**-1.37565
print(c1)
d = -0.9

v_dummy_df['R_w'] = c1*c2*c5*dsp*g*rho_sea*math.exp(m_1*Fr_n**d+m_2*math.cos(lambda_const*Fr_n**-2))
print(v_dummy_df['R_w'])

0.04360273752148408
0.6739118418119685
0.9592
0.7737872716316899
1.302542119617646
-2.061826835604456
-0.13919818956744992
19.867152251684907
1.6394657947130131
0    91773.557081
1    91773.557081
2    91773.557081
3    91773.557081
4    91773.557081
Name: R_w, dtype: float64


In [259]:
print(A_bt)
print(T_fwd)

11.24627236597615
5.85


In [260]:
# resistance due to bulbous bow
# use V = 10 m/s

P_b = 0.56*math.sqrt(A_bt)/(T_fwd-1.5*h_b)
print(P_b)
v_dummy_df['F_ni'] = v_dummy_df['dummy_vel_ms'] / np.sqrt(g*(T_fwd-h_b-0.25*math.sqrt(A_bt))+0.15*v_dummy_df['dummy_vel_ms']**2)
v_dummy_df['R_b'] = 0.11 * np.exp(-3*P_b**-2)*v_dummy_df['F_ni']**3*A_bt**1.5*rho_sea*g/(1+v_dummy_df['F_ni']**2)
print(v_dummy_df['R_b'])

3.210232295573174
0    39665.078556
1    41570.874930
2    43333.796507
3    44963.748566
4    46470.525453
Name: R_b, dtype: float64


In [261]:
#Calculate Transom Resistance 
# use v = 10

F_nT = 10 / math.sqrt(2*g*A_t/(B+B*C_wp))
print(F_nT)
c6 = 0
R_tr = 0.5*rho_sea*10**2*A_t*c6
print(f"R_tr = {R_tr:0.4f} N")

5.5434260311174
R_tr = 0.0000 N


In [262]:
# Model ship correlation resistance

c4 = T_fwd/lwl
print(c4)
# since c4>0.04, use c4 as c4 = 0.04
c4 = 0.04
ca = 0.006*(lwl+100)**-0.16 - 0.00205 + 0.003*math.sqrt(lwl/7.5)*C_b**4*c2*(0.04-c4)
print(ca)
v_dummy_df['R_a'] = 0.5*rho_sea*v_dummy_df['dummy_vel_ms']**2*S_bh*ca
print(v_dummy_df['R_a'])


0.04040055248618784
0.00043852165454709695
0     68035.836645
1     76275.470840
2     84985.941276
3     94167.247951
4    103819.390866
Name: R_a, dtype: float64


In [263]:
# Calculate Rtot

v_dummy_df['R_tot'] = v_dummy_df['R_f'] + v_dummy_df['Rapp'] + v_dummy_df['R_w'] + v_dummy_df['R_b'] + v_dummy_df['R_a'] # in kN

In [264]:
v_dummy_df.head()

,dummy_vel,dummy_vel_ms,Re,cf,R_f,Rapp,R_w,F_ni,R_b,R_a,R_tot
0,17,8.745576,1.073186e+09,0.000920,172913.623194,5046.840273,91773.557081,1.708717,39665.078556,68035.836645,377.434936
1,18,9.260021,1.136314e+09,0.000915,192793.359170,5627.071317,91773.557081,1.763078,41570.874930,76275.470840,408.040333
2,19,9.774467,1.199443e+09,0.000910,213700.189821,6237.280235,91773.557081,1.813374,43333.796507,84985.941276,440.030765
3,20,10.288913,1.262572e+09,0.000905,235629.204829,6877.323709,91773.557081,1.859904,44963.748566,94167.247951,473.411082
4,21,10.803358,1.325700e+09,0.000901,258575.778420,7547.066726,91773.557081,1.902950,46470.525453,103819.390866,508.186319


In [265]:
v_dummy_df2=v_dummy_df.drop(['cf','Re','F_ni'],axis=1)

In [266]:
# Calculate Efficiencies

# First n_h
# Approximate diameter size from Bertram

D = 0.215*16 #m
# D = 8
print(D)
print(C_b)
print(k1_const)
print(B)
print(T_n)

v_dummy_df['cv'] = k1_const*v_dummy_df['cf'] + ca
# print(v_dummy_df['cv'])
v_dummy_df['w'] = 0.3095 * C_b + 10*v_dummy_df['cv']*C_b - (0.23*D)/math.sqrt(B*T_n) 
# print(v_dummy_df['w'])
v_dummy_df['t'] = 0.325*C_b - 0.1885*D/math.sqrt(B*T_n)
# print(v_dummy_df['t'])
v_dummy_df['eff_h'] = (1-v_dummy_df['t']) / (1-v_dummy_df['w'])
print(v_dummy_df['eff_h'])

# Then n_r
v_dummy_df['eff_r'] = 0.9737 + 0.111*(C_p - 0.225*lcb) - 0.06325*lcb
print(v_dummy_df['eff_r'])

v_dummy_df['eff_s'] = 0.99 # Set according to holtrop mennen and man
v_dummy_df['eff_o'] = 0.75 # Approx wageningen kristensen

v_dummy_df2['eff_tot'] = v_dummy_df['eff_h']* v_dummy_df['eff_r']* v_dummy_df['eff_s']*v_dummy_df['eff_o']
# print(v_dummy_df2)
v_dummy_df2['P_b'] = (v_dummy_df2['R_tot']/1e3 * v_dummy_df2['dummy_vel_ms'])/v_dummy_df2['eff_tot'] # in kW
v_dummy_df2.head()
# Unit P_b is KW

3.44
0.6451363786295532
1.2118857645707746
24.5
5.85
0    0.986099
1    0.986053
2    0.986011
3    0.985971
4    0.985933
Name: eff_h, dtype: float64
0    1.048474
1    1.048474
2    1.048474
3    1.048474
4    1.048474
Name: eff_r, dtype: float64


,dummy_vel,dummy_vel_ms,R_f,Rapp,R_w,R_b,R_a,R_tot,eff_tot,P_b
0,17,8.745576,172913.623194,5046.840273,91773.557081,39665.078556,68035.836645,377.434936,0.767670,4.299878
1,18,9.260021,192793.359170,5627.071317,91773.557081,41570.874930,76275.470840,408.040333,0.767634,4.922216
2,19,9.774467,213700.189821,6237.280235,91773.557081,43333.796507,84985.941276,440.030765,0.767601,5.603256
3,20,10.288913,235629.204829,6877.323709,91773.557081,44963.748566,94167.247951,473.411082,0.767570,6.345851
4,21,10.803358,258575.778420,7547.066726,91773.557081,46470.525453,103819.390866,508.186319,0.767541,7.152870


In [267]:
SFOC = 169.4 # g/kWh
v_dummy_df2['FOC/h'] = (v_dummy_df2['P_b'] * SFOC)/1e6 # get FOC t/h
v_dummy_df2.head()


,dummy_vel,dummy_vel_ms,R_f,Rapp,R_w,R_b,R_a,R_tot,eff_tot,P_b,FOC/h
0,17,8.745576,172913.623194,5046.840273,91773.557081,39665.078556,68035.836645,377.434936,0.767670,4.299878,0.000728
1,18,9.260021,192793.359170,5627.071317,91773.557081,41570.874930,76275.470840,408.040333,0.767634,4.922216,0.000834
2,19,9.774467,213700.189821,6237.280235,91773.557081,43333.796507,84985.941276,440.030765,0.767601,5.603256,0.000949
3,20,10.288913,235629.204829,6877.323709,91773.557081,44963.748566,94167.247951,473.411082,0.767570,6.345851,0.001075
4,21,10.803358,258575.778420,7547.066726,91773.557081,46470.525453,103819.390866,508.186319,0.767541,7.152870,0.001212
